In [ ]:
import pandas as pd
import re

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [ ]:
ratingsFile = "./ml-100k-csv/merged_ratings.csv"
ratings1 = pd.read_csv(ratingsFile)

ratings2 = ratings1[["user_id", "movie_id", "rating", "title"]]

In [ ]:
print(ratings2.head())

In [ ]:
def separate_title_year(df, title_column="title"):
    """
    Separates the year from movie titles in a DataFrame.

    Parameters:
    df (DataFrame): DataFrame containing movie data
    title_column (str): Name of the column containing movie titles with years

    Returns:
    DataFrame: DataFrame with separate title and year columns
    """
    # Create a copy of the DataFrame to avoid modifying the original
    result_df = df.copy()

    # Function to extract year from title
    def extract_year(title):
        # Look for a 4-digit number in parentheses at the end of the string
        year_match = re.search(r"\((\d{4})\)$", title)
        if year_match:
            # Extract the year (without parentheses)
            return int(year_match.group(1))
        else:
            return None

    # Function to clean title by removing the year in parentheses
    def clean_title(title):
        # Remove the year in parentheses at the end
        return re.sub(r"\s*\(\d{4}\)$", "", title).strip()

    # Apply the extraction functions to create new columns
    result_df["movie_year"] = result_df[title_column].apply(extract_year)
    result_df["movie_title"] = result_df[title_column].apply(clean_title)

    return result_df


# Process the sample DataFrame
clean_ratings = separate_title_year(ratings2)
ratings = clean_ratings[["user_id", "movie_id", "movie_title", "movie_year", "rating"]]
ratings = ratings.sort_values(by="user_id", axis=0, inplace=False)

# Function to reformat movie titles
ratings["movie_title"] = ratings["movie_title"].str.replace(
    r"^(.*), (The|A|An|La|Le|Les|El|Los|Las|Der|Die|Das|Une|Un|Una|Uno)$",
    r"\2 \1",
    regex=True,
)


# Display the result with the new columns
print(ratings.head())

# Working with movie_records.ipynb AND this user data

In [ ]:
movies = pd.read_csv(
    "/Users/nakiyahdhariwala/share_community/large_files/movie_records.csv",
    dtype={"theme": str},
)

print(movies.head())

In [ ]:
# Create a function to standardize titles
def standardize_title(title):
    if isinstance(title, str):
        # Convert to lowercase
        title = title.lower()
        # Remove special characters and extra spaces
        import re

        title = re.sub(
            r"[^\w\s]", "", title
        )  # Remove non-alphanumeric characters except spaces
        title = re.sub(
            r"\s+", " ", title
        )  # Replace multiple spaces with a single space
        title = title.strip()  # Remove leading/trailing spaces
    return title


# Apply the standardization to both dataframes
movies["movie_name"] = movies["name"].apply(standardize_title)
ratings["movie_name"] = ratings["movie_title"].apply(standardize_title)

# Now merge on the standardized titles
df_inner = pd.merge(movies, ratings, on="movie_name", how="inner")
df_outer = pd.merge(movies, ratings, on="movie_name", how="outer")
df_left = pd.merge(movies, ratings, on="movie_name", how="inner")
df_right = pd.merge(movies, ratings, on="movie_name", how="inner")

# For simplicity, I am choosing to keep the inner join

In [ ]:
print(df_inner.head())

In [ ]:
print(df_inner.columns)

final_df = df_inner[
    ["user_id", "id", "name", "date", "description", "genre", "theme", "rating"]
]
print(final_df.head())

In [ ]:
final_df.to_csv("Movie_users.csv", index=False)

In [ ]:
print(df_inner[df_inner["user_id"] == 798])